# Week 11 JupyterNotebook

Week 10 lecture focused on:

1) OOP
2) Class Attributes and Class Methods
3) Class Statement
4) More Realistic Examples
5) Steps of Creating Class Person and Its Instances 
6) Class Coding Details
7) Revisit namespace and score resolution

This lecture focuses on:
1) ``__iter__``, ``__next__``, ``__len__``, ``__max__``, ``__eq__``, ``__lt__``, ``__gt__``
2) Three ways to combine classes:
   
   -- Inheritance: “Is-A” relationships
   
   -- Composition: “Has-A” relationships
   
   -- Delegation: “Like-A” relationships
3) Revisit ``getattr()`` and ``__getattrt__()``

In [ ]:
class Squares:
    def __init__(self, start, stop):  
        self.start = start
        self.stop  = stop

    def __iter__(self): 
        #print("__iter__")
        return self             # Returns an iterator object, making self an iterator

    def __next__(self):         # Returns the next value on each iteration   
        if self.start == self.stop: 
            raise StopIteration
        #print("__next__", end = '')
        value = self.start ** 2
        self.start += 1
        return value

obj = Squares(0, 4) # Make an instance obj: obj.start = 0, obj.stop = 4
for i in obj:       # The for loop triggers __iter__ once and then the __next__
    print(i)

#### When ``__iter__()`` is a generator (uses yield), Python automatically builds the iterator. In this case, the generator already has a ``__next__()`` method.

In [ ]:
class Squares:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __iter__(self):
        #print("__iter__")
        while self.start < self.end:
            #print("__next__", end = '')
            yield self.start ** 2
            self.start += 1

obj = Squares(0, 4)  
for i in obj:        
    print(i)

In [ ]:
"""
Compute or stream data on the fly instead of storing everything in memory.
"""

class Fibonacci:
    def __init__(self, limit):
        self.limit = limit

    def __iter__(self):
        a, b = 0, 1
        while a <= self.limit:
            yield a        # It doesn’t store all Fibonacci numbers in memory at once -- it streams them one by one
            a, b = b, a + b

print([fib for fib in Fibonacci(8)])

In [ ]:
"""
Trace the memory usage for executing a block of code
"""

import tracemalloc

tracemalloc.start()

class Fibonacci:
    def __init__(self, limit):
        self.limit = limit

    def __iter__(self):
        a, b = 0, 1
        while a <= self.limit:
            yield a
            a, b = b, a + b

print([fib for fib in Fibonacci(8)])

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage: {current / 1024:.1f} KB; Peak: {peak / 1024:.1f} KB")
tracemalloc.stop()

### ``__iter__`` can also return a NEW iterator, e.g., a list iterator. Then the ``__next__`` is built-in. So ``__iter__`` deligatres iterations to the list iterator

In [ ]:
""" 
Class represents a collection of data, making it iterable lets users to use 
for loops, comprehensions, and built-ins like len().
"""

class SensorLog:  
    def __init__(self, readings, loc = 'Hoboken'):
        self.readings = readings
        self.loc = loc

    def __iter__(self):
        return iter(self.readings)  # Returns a NEW list iterator.
        
    def __len__(self):
        return len(self.readings)    

temps = SensorLog([20, 22, 21, 19]) 
print(f"{len(temps)} collections at {temps.loc}")
print(f"Readings greater than 20: {[t for t in temps if t > 20]}")

In [ ]:
class Student:
    def __init__(self, name, grade):
        self.name, self.grade = name, grade   

    def __eq__(self, other):
        return self.grade == other.grade

    def __lt__(self, other):
        return self.grade < other.grade

    def __gt__(self, other):
        return self.grade > other.grade

In [ ]:
s1 = Student("Bob", 98)
s2 = Student("Sue", 90)
s3 = Student("Pat", 85)

In [ ]:
s1 == s2

In [ ]:
s1 < s3

In [ ]:
s2 > s3

## 3 ways to combine classes

### Inheritance: “Is-A” Relationship

In [ ]:
# employees.py <- save it to your Notebook folder

"""
Inheritance: “Is-A” Relationship
"""

class Employee:
    def __init__(self, name, salary = 0):
        self.name   = name
        self.salary = salary

    def giveRaise(self, percent):
        self.salary += self.salary * percent

    def work(self):
        print(self.name, 'does stuff')

    def __repr__(self):
        return (f'{self.__class__.__name__}: {self.name}, salary {self.salary:,.2f}')

class Server(Employee):    # Inherits from Employee
    def __init__(self, name):
        super().__init__(name, 40000)

    def work(self):
        print(self.name, 'interfaces with customer')
        
class Chef(Employee):      # Inheritance
    def __init__(self, name):
        super().__init__(name, 50000)

    def work(self):
        print(self.name, 'makes food')

class PizzaRobot(Chef):    # Inherits from Chef, which inherits from Employee
    def __init__(self, name):
        super().__init__(name)

    def work(self):
        print(self.name, 'makes pizza')

if __name__ == '__main__':
    pr = PizzaRobot('Pat')       # Make a robot named Pat
    print(pr)                    # Run inherited __repr__
    pr.work()                    # Run type-specific action
    pr.giveRaise(0.10)           # Give pat a 10% raise
    print(pr, '\n')

    ecsp = [Employee, Server, Chef, PizzaRobot]  # A list of class objects (Classes are objects)
    for clses in ecsp:
        obj = clses(clses.__name__)  # Make 4 instances; Notice all constructors expect a "name"
        obj.work()

### Composition: “Has-A” Relationship

In [ ]:
"""
Composition: “Has-A” Relationship
one object "owns" another object
"""

class CPU:
    def __init__ (self): 
        print("CPU is processing data...")

class Memory:
    def __init__(self): 
        print("Memory is loading programs...")  

class Computer:
    def __init__(self, cpu = '', memory = ''):
        self.cpu = CPU()          # Computer has-a CPU.  self.cpu refers to the CPU instance contained inside the Computer
        self.memory = Memory()    # Computer has-a Memory
                                  # Creates CPU and Memory instances and stores them inside the Computer's attributes
        #print(self.cpu)
        #print(self.memory)

my_computer = Computer()
print("My computer is working...") 

In [ ]:
"""
Composition: “Has-A” Relationship
one object "owns" another object
"""

class Person:
    def __init__(self, name, job = None, pay = 0):
        self.name = name
        self.job  = job
        self.pay  = pay

    def giveRaise(self, percent):
        self.pay = self.pay * (1 + percent)

    def __repr__(self):
        return f'{self.__class__.__name__}: {self.name} ${self.pay:,.2f}'

class Manager(Person):                           # Inheritance
    def __init__(self, name, pay):               # Redefine constructor
        super().__init__(name, 'mgr', pay)       # Run original with 'mgr'

    def giveRaise(self, percent, bonus = .10):
        super().giveRaise(percent + bonus)       # Person.giveRaise(self, percent + bonus)

class Department:
    def __init__(self, *args):                  # Take arbitrary positional arguments and pack them into a tuple.
        self.members = args                     # Store the tuple inside an attribute "members"
        #print(f"self.members are: {self.members}")

    def showAll(self):                          # Display all nested instance objects
        for pm in self.members:
            pm.giveRaise(0.10)
            print(pm)

if __name__ == '__main__':
    pat = Person('Pat Jones', job = 'dev', pay = 90000)
    mike = Manager('Mike White', 100000)
    people = Department(pat, mike)     # Creates a Department instance that contains instances of the other two classes
    people.showAll()                   

### Delegation: “Like-A” Relationship

In [ ]:
"""
Delegation: “Like-A” Relationship
one object asks another object of a different class to do something
"""

class Engine:
    def startE(self):
        print("Engine starting...")

class Car:
    def startC(self, argu):
        #print(f"{type(engine)}")   
        argu.startE()  # Car delegates to argu.startE(), 
                         # Here argu is an instance of Engine  
                         # ==> e.startE()

e = Engine()
c = Car()
c.startC(e)     # An instance of class Engine is passed to Car.startC(c, e)

### Delegation with Inheritance

In [ ]:
"""
Delegation with Inheritance:
one object asks an inherited object to do something
"""

class A:
    def taskA(self):
        #print(f"Location of self in Class A: {hex(id(self))}")
        print("taskA() is working")

class B(A):   # B inherits from A
    def taskB(self):
        print("taskB() delegats to taskA()")
        super().taskA() # super() serves a special proxy object that knows how to delegate method calls to 
                        # the next class in the method resolution order (MRO) -- Class A.
                        # super.taskA() <-> A.taskA(b_obj)

b_obj = B()
#print(f"The b_obj location: {hex(id(b_obj))}")
b_obj.taskB()

### Delegation with Compistion

In [ ]:
"""
Delegation with Compistion:
one owns another one and ask the other one to do something
"""

class Printer:          # class Printer knows how to print
    def __init__(self): 
        print('Printer is ready.')

    def print(self, message): 
        print(f"{message} is printed.")     

class Staff:            
    def __init__(self, printer = ''):  # Staff "has-a" printer (Composition)
        self.printer = Printer()       # Creates a Printer instance object and stores it inside Staff's attribute printer
                                       # self.printer refers to the Printer instance contained inside the Staff

s = Staff()                            # s.printer in an instance of class Printer
s.printer.print("Staff's message")     # => Printer.print(s.printer, "Staff's messag") 
#Printer.print(s.printer, "Staff's messag")   

In [ ]:
"""
Delegation with Compistion:  
one owns another one and ask the other one to do something
"""

class CPU:
    def process(self):
        print("CPU is processing data...")

class Memory:
    def load(self):
        print("Memory is loading programs...")

class Storage:
    def read(self):
        print("Storage is reading files...")

class Computer:
    def __init__(self):
        self.cpu = CPU()          # Computer has-a CPU
        self.memory = Memory()    # Computer has-a Memory
        self.storage = Storage()  # Computer has-a Storage  

    def start(self):
        print("Starting computer...")
        self.cpu.process()        # Delegations
        self.memory.load()        
        self.storage.read()

my_computer = Computer()
my_computer.start()    

In [ ]:
from employees import PizzaRobot, Server

class Customer:
    def __init__(self, name):
        self.name = name

    def order(self, server):
        print(self.name, 'orders from', server)

    def pay(self, server):
        print(self.name, 'pays for item to', server)

class Oven:
    def bake(self):
        print('Oven bakes')

class PizzaShop:
    def __init__(self):
        self.server = Server('Sam')         # Contain other instance objects (Composition)
        self.chef   = PizzaRobot('Pat')      
        self.oven   = Oven()

    def order(self, name):
        customer = Customer(name)           # Contain an instance object of Customer (Composition)
        customer.order(self.server)         # Customer orders from server
        self.chef.work()
        self.oven.bake()
        customer.pay(self.server)

if __name__ == '__main__':
    scene = PizzaShop()                     # Make the composite
    scene.order('Sue')                      # Simulate Sue's order

### Revisit ``getattr()`` and ``__getattr__()``

#### ``getattr()`` could return:

1) an attribute's value
2) a bound method
3) automatically triggers the object’s ``__getattr__()`` method (if defined) when accessing a missing attribute.

This makes Python flexible and dynamic.

In [ ]:
# x is an attribute

class Real:
    def __init__(self):
        self.greet = "Hello!"  # greet is an attribute

obj = Real()
x = getattr(obj, "greet")  # Retrieves a string. Find the attribute in obj.__dict__
print(x)

In [ ]:
# x is a method

class Real:
    def greet(self):
        return "Hello!"

obj = Real()
x = getattr(obj, "greet")   # Access an instance's method (greet) <=> obj.greet
                            # Python automatically binds Real.greet with the instance (obj) and
                            # returns the bound method and assigns it to x 
x()   # Because x is bound to obj, Python automatically passes obj to the method when calling x()
      # x() <=>  Real.greet(obj)

#print(x.__func__ is Real.greet)    # x.__func__: the underlying function object defined in the class
#print(x.__self__ is obj)           # x.__self__: the instance that the bound method is bound to
#print(id(x.__self__) == id(obj))  

In [ ]:
class Real:
    def __init__(self):
        self.size = 10
    
    def __getattr__(self, name):
        #print(f"__getattr__ triggered for: {name}")
        return "NotFound"

obj = Real()
x = getattr(obj, "size")   # <==> obj.size. Found it in obj.__dict__
print(f"size is {x}")

x = getattr(obj, "color")  # obj.color? Not found => triggers __getattr__()
print(f"color is {x}")

In [ ]:
class Person:
    def __init__(self, name, job = None, pay = 0):
        self.name = name
        self.job = job
        self.pay = pay

    def lastName(self):
        print(f"self in Person.lastNmae(): {id(self)}") 
        return self.name.split()[-1]

class Manager:
    def __init__(self, name, pay = 0):
        self.person = Person(name, 'mgr', pay)  # Composition. self.person refers to the Person instance contained inside the Manager
        print(f"self.person in Manager.__init__: {id(self.person)}")

    def __getattr__(self, attr):           # Only called as a fallback — when normal attribute lookup fails
        #print(f"self in __getattr__: {id(self)}")   # self == obj
        return getattr(self.person, attr)  # __getattr__ returns getattr(self.person, "lastName") => getattr(obj.person, "lastName")
                                           # Delegate attribute access to obj.person.lastName
                                           # obj.person is bound to an instance object of class Person
                                                 # => getattr binds Person.lastName to obj.person
                                           # obj.person.lastName() => eventually executes Person.lastName(obj.person)
                                           # ... ...
                                           # obj.pay => eventually returns self.person.pay
if __name__ == '__main__':
    obj = Manager('Mike Jones', pay = 50000)  # Creates one Manager instance (obj) AND 
    #print(f"obj: {id(obj)}")                  # one Person instance (obj.person) => self.person

    print(obj.lastName(), obj.pay)   # obj.lastName => Python looks for lastName in obj.__dict__ and class Manager.__dict__, not found. 
                                     # So triggers Manager.__getattr__(obj, "lastName"). Now, self is the Manager instance (obj)
                                     # ... ...
                                     # obj.pay -> Python looks for pay in Manager, not found. 
                                     # Again, triggers Manager.__getattr__(obj, "pay").   

## After class exercise

### Let’s look at a class where the same attribute name (greet) is first assigned a string, then later replaced by a method, all at runtime.

### This demonstrates how Python doesn't care whether something is an attribute or a method — it just treats "greet" as a key in the object's namespace (``__dict__``), and what’s stored under that key determines its behavior.

### A class object is passed to a function as an argument